In [ ]:
import jax
import jax.numpy as jnp

In [ ]:
from rubix.pipeline import linear_pipeline as ltp
from rubix.pipeline import transformer as rtr
from rubix.utils import read_yaml

In [ ]:
class Add(rtr.TransformerFactoryBase):
    def __init__(self, value: float = 0.0):
        self.s = value

    def create(self) -> jnp.array:
        @jax.jit
        def f(a):
            s = self.s
            return a + s

        return f


class Multiply(rtr.TransformerFactoryBase):
    def __init__(self, factor: float = 0):
        self.s = factor

    def create(self) -> jnp.array:
        @jax.jit
        def f(a):
            s = self.s
            return a * s

        return f


class Divide(rtr.TransformerFactoryBase):
    def __init__(self, divisor: float = 0):
        self.s = divisor

    def create(self) -> jnp.array:
        @jax.jit
        def f(a):
            s = self.s
            return a / s

        return f


In [ ]:
cfg = {
    "Transformers": {
        "A": {
            "name": "Add",
            "depends_on": "B",
            "args": {
                "value": 3.0,
            },
        },
        "X": {
            "name": "Multiply",
            "depends_on": "A",
            "args": {
                "factor": 2,
            },
        },
        "Z": {
            "name": "Divide",
            "depends_on": "X",
            "args": {
                "divisor": 4,
            },
        },
        "B": {
            "name": "Multiply",
            "depends_on": "C",
            "args": {
                "factor": 2,
            },
        },
        "C": {
            "name": "Add",
            "depends_on": None,
            "args": {
                "value": 4,
            },
        },
    }
}

In [ ]:
read_cfg = read_yaml("./demo.yml")

In [ ]:
read_cfg

In [ ]:
tp = ltp.LinearTransformerPipeline(cfg)

In [ ]:
for classname in [Add, Multiply, Divide]: 
    tp.register_transformer(classname)

In [ ]:
tp.transformers

In [ ]:
tp.assemble()

In [ ]:
tp.pipeline

In [ ]:
tp.expression

In [ ]:
jax.jit(tp.expression)

In [ ]:
x = jnp.array([3, 2, 1])

In [ ]:
res = tp.apply(x)

In [ ]:
res

In [ ]:
 res_hand = ((((x + 4) * 2) + 3) * 2) / 4

In [ ]:
res_hand

In [ ]:
all(res == res_hand)